In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def generation_data(noise_level):
    x = np.asarray(np.linspace(1,10,100),dtype=np.float32)
    x.shape = (100,1)
    y = np.sin(x)+noise_level*np.random.rand(100,1)
    y = np.asarray(y,dtype=np.float32)
    return x,y

In [ ]:
in_data,out_data = generation_data(noise_level=0.1)

In [ ]:
plt.plot(in_data,out_data)

In [ ]:
import torch
from torch import nn, optim
from torch.autograd import Variable

In [ ]:
# Linear Regression Model
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear1 = nn.Linear(1, 10)  # input and output is 1 dimension
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(10,10)
        self.linear3 = nn.Linear(10,1)

    def forward(self, x):
        out_layer1 = self.relu(self.linear1(x))
        out_layer2 = self.relu(self.linear2(out_layer1))
        out = self.linear3(out_layer2)
        return out

In [ ]:
model = LinearRegression()
model = nn.DataParallel(model)
model.cuda()
# 定义loss和优化函数
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1e-5)
x_train = torch.from_numpy(in_data)
y_train = torch.from_numpy(out_data)

In [ ]:
# 开始训练
num_epochs = 10000
batch_size = 5
batch_num_max = 20

for epoch in range(num_epochs):
    
    for batch_num in range(batch_num_max):
        x_train = torch.from_numpy(in_data[batch_num*batch_size:batch_num*batch_size+batch_size,:])
        y_train = torch.from_numpy(out_data[batch_num*batch_size:batch_num*batch_size+batch_size,:])
        inputs = Variable(x_train).cuda()
        target = Variable(y_train).cuda()

        # forward
        out = model(inputs)
        loss = criterion(out, target)
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 20 == 0:
        print('Epoch[{}/{}], loss: {:.6f}'
              .format(epoch+1, num_epochs, loss.data[0]))


In [ ]:
model.eval()
predict = model(Variable(x_train))
predict = predict.data.numpy()

In [ ]:
plt.plot(in_data,out_data,in_data,predict)